## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Lavrentiya,RU,2022-05-05 18:25:06,65.5833,-171.0000,14.77,90,40,4.38
1,1,Anuradhapura,LK,2022-05-05 18:25:06,8.3565,80.4173,77.52,96,99,6.55
2,2,Cape Town,ZA,2022-05-05 18:23:58,-33.9258,18.4232,60.12,89,20,5.75
3,3,Krasnouralsk,RU,2022-05-05 18:25:07,58.3638,60.0407,36.43,97,100,14.63
4,4,Mount Isa,AU,2022-05-05 18:25:07,-20.7333,139.5000,58.77,41,99,14.97


In [3]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     545 non-null    int64  
 1   City        545 non-null    object 
 2   Country     542 non-null    object 
 3   Date        545 non-null    object 
 4   Lat         545 non-null    float64
 5   Lng         545 non-null    float64
 6   Max Temp    545 non-null    float64
 7   Humidity    545 non-null    int64  
 8   Cloudiness  545 non-null    int64  
 9   Wind Speed  545 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 42.7+ KB


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_min_temp = int(input("What is the minimum temperature you would like?: "))
user_max_temp = int(input("What is the maximum temperature you would like?: "))

What is the minimum temperature you would like?: 45
What is the maximum temperature you would like?: 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
mask = (city_data_df["Max Temp"] <= user_max_temp) & (city_data_df["Max Temp"] >= user_min_temp)
fil_city_data_df = city_data_df.loc[mask]

In [10]:
# 4a. Determine if there are any empty rows.
fil_city_data_df.count()

City_ID       422
City          422
Country       419
Date          422
Lat           422
Lng           422
Max Temp      422
Humidity      422
Cloudiness    422
Wind Speed    422
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
fil_city_data_df.dropna()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Anuradhapura,LK,2022-05-05 18:25:06,8.3565,80.4173,77.52,96,99,6.55
2,2,Cape Town,ZA,2022-05-05 18:23:58,-33.9258,18.4232,60.12,89,20,5.75
4,4,Mount Isa,AU,2022-05-05 18:25:07,-20.7333,139.5000,58.77,41,99,14.97
5,5,Kahului,US,2022-05-05 18:25:08,20.8947,-156.4700,75.85,80,100,13.80
6,6,Kang,BW,2022-05-05 18:25:08,-23.6752,22.7876,65.73,36,5,4.59
...,...,...,...,...,...,...,...,...,...,...
538,538,Necochea,AR,2022-05-05 18:40:07,-38.5473,-58.7368,67.53,51,100,11.27
539,539,Primo Tapia,MX,2022-05-05 18:40:08,32.2200,-116.9172,78.94,58,15,11.36
540,540,Puerto Wilches,CO,2022-05-05 18:40:08,7.3483,-73.8960,84.76,68,100,2.73
542,542,Cabatuan,PH,2022-05-05 18:40:09,10.8794,122.4860,77.54,85,77,7.58


In [15]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = fil_city_data_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Anuradhapura,LK,77.52,8.3565,80.4173,
2,Cape Town,ZA,60.12,-33.9258,18.4232,
4,Mount Isa,AU,58.77,-20.7333,139.5000,
5,Kahului,US,75.85,20.8947,-156.4700,
6,Kang,BW,65.73,-23.6752,22.7876,
7,Mataura,NZ,52.48,-46.1927,168.8643,
9,Sao Jose Da Coroa Grande,BR,82.24,-8.8978,-35.1478,
11,Hithadhoo,MV,83.84,-0.6000,73.0833,
12,Bonthe,SL,84.16,7.5264,-12.5050,
14,Ushuaia,AR,49.66,-54.8000,-68.3000,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Found Hotel")
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Found Hotel
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Found Hotel
Hotel not found... skipping.
Hotel not found... skipping.
Found Hotel
Hotel not found... skipping.
Found Hotel
Found Hotel
Hotel not found... skipping.
Found Hotel
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Found Hotel
Hotel not found... skipping.
Found Hotel
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Found Hotel
Found Hotel
Hotel not 

In [24]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Anuradhapura,LK,77.52,8.3565,80.4173,
2,Cape Town,ZA,60.12,-33.9258,18.4232,
4,Mount Isa,AU,58.77,-20.7333,139.5000,
5,Kahului,US,75.85,20.8947,-156.4700,
6,Kang,BW,65.73,-23.6752,22.7876,
7,Mataura,NZ,52.48,-46.1927,168.8643,
9,Sao Jose Da Coroa Grande,BR,82.24,-8.8978,-35.1478,
11,Hithadhoo,MV,83.84,-0.6000,73.0833,
12,Bonthe,SL,84.16,7.5264,-12.5050,
14,Ushuaia,AR,49.66,-54.8000,-68.3000,Albatross Hotel


In [25]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
14,Ushuaia,AR,49.66,-54.8000,-68.3000,Albatross Hotel
23,Rikitea,PF,78.24,-23.1203,-134.9692,People ThankYou
27,Butaritari,KI,81.86,3.0707,172.7902,Isles Sunset Lodge
29,Kapaa,US,77.32,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
30,Palmer,US,70.77,42.1584,-72.3287,Wedgewood Motel
32,Castro,BR,69.42,-24.7911,-50.0119,CHACARA BAILLY
39,Luderitz,NaN,66.85,-26.6481,15.1594,LÜDERITZ NEST HOTEL
41,Port Hardy,CA,46.40,50.6996,-127.4199,Airport Inn-Port Hardy
47,Sambava,MG,76.17,-14.2667,50.1667,Melrose
48,Dawei,MM,77.13,14.0833,98.2000,Hotel Dawei


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))